In [ ]:
# -*- coding: utf-8 -*-
!pip install matplotlib
!pip install networkx
!pip install numpy
!pip install haversine
!pip install cartopy
!pip install geopandas
!pip install datetime
!pip install openpyxl
!pip install ortools 
!pip install pyro-ppl 
!pip install fiona shapely pyproj rtree
!pip install --upgrade fiona geopandas
   
import os
import numpy as np
import networkx as nx
import ortools
from ortools.linear_solver import pywraplp
from haversine import haversine
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import geopandas as gpd
import fiona
import pandas as pd
import matplotlib.cm as cm
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
from datetime import datetime
from datetime import timedelta
from openpyxl import load_workbook
import torch
import pyro
import pyro.distributions as dist
from datetime import datetime
from datetime import timedelta
from openpyxl import load_workbook
from sklearn.model_selection import train_test_split
from pyro.infer import Predictive
from pyro.infer.autoguide import AutoDiagonalNormal
from pyro.infer import MCMC, NUTS

# copy the repository

repo_url = "https://github.com/rimchmielowitz/Pricing_ML.git"
repo_name = "Pricing_ML"

if not os.path.exists(repo_name):
    !git clone --recurse-submodules {repo_url}
    print("✅ Repository wurde geklont.")
else:
    print("✅ Repository existiert bereits.")

# change to repository folder
os.chdir(repo_name)

# SHAPE_RESTORE_SHX aktivieren
os.environ["SHAPE_RESTORE_SHX"] = "YES"

#paths of Berlin instances

data_folder = os.path.join(os.getcwd(), "data_Berlin")
data1 = os.path.join(data_folder, "TWD.csv")
data2 = os.path.join(data_folder, "districts.csv")

M = 10000
time_format_output='%H:%M'

#model
def pricing_model(n, m, A, A_, H, WTP, ETP, R, d2, V, d, t, s, l, a, b, M):
    
    # Initializing OR-Tools Solver (MIP-Optimization)
    solver = pywraplp.Solver.CreateSolver('SCIP')

    if not solver:
        print("Solver konnte nicht erstellt werden!")
        return None

    # Definitiion of variables
    x = {}
    for (i, j) in E:
        for h in H:
            x[i, j, h] = solver.IntVar(0, 1, f'x[{i},{j},{h}]')

    S = {}
    for i in V:
        for h in H:
            S[i, h] = solver.IntVar(0, solver.infinity(), f'S[{i},{h}]')

    L = {}
    for i in V:
        for h in H:
            L[i, h] = solver.IntVar(0, solver.infinity(), f'L[{i},{h}]')

    z = {}
    for i in A:
        z[i] = solver.IntVar(0, 1, f'z[{i}]')

    p = {}
    for (i, j) in E:
        for h in H:
            p[i, j, h] = solver.NumVar(0, solver.infinity(), f'p[{i},{j},{h}]')

    c = {}
    for (i, j) in E:
        for h in H:
            c[i, j, h] = solver.NumVar(0, solver.infinity(), f'c[{i},{j},{h}]')

    # objective function: Maximize the profit (price - costs)
    solver.Maximize(
        solver.Sum(p[i, j, h] * d2[i] - c[i, j, h] * d[i, j] for (i, j) in E for h in H if i != j)
    )

    # Constraints
    for h in H:
        for i in A:
            for j in pickup_delivery:
                if i != j:
                    solver.Add(p[i, j, h] <= WTP[i] / d2[i] + (1 - x[i, j, h]) * M) #Price_not_greater_than_WTP(02)

    for h in H:
        for i in origin_pickup_delivery[h]:
            for j in pickup_delivery:
                if i != j:
                    solver.Add(c[i, j, h] >= ETP[h] * x[i, j, h]) #Compensation_not_less_than_ETP(03)

    for h in H:
        for i in A:
            for j in pickup_delivery:
                if i != j:
                    solver.Add(p[i, j, h] >= c[i, j, h]) #Price_bigger_Compensation(04)

    for i in A:
        solver.Add(solver.Sum(x[i, j, h] for j in pickup_delivery for h in H if i != j) + z[i] == 1) #i_in_requestbank_or_picked_up(05)

    for h in H:
        for i in A:
            solver.Add(
                solver.Sum(x[i, j, h] for j in Vh[h] if (i, j, h) in x and i != j) - solver.Sum(x[j, n+i, h] for j in Vh[h] if (j, n+i, h) in x and i != n+i) == 0
            ) #matched_picked_delivered_same_courier(06) 

    for h in H:
        solver.Add(solver.Sum(x[origin[h], j, h] for j in pickup_destination[h]) == 1) #courier_starts_origin_to_pickup_or_destination(07)

    for h in H:
        solver.Add(solver.Sum(x[i, tau_[h], h] for i in delivery_origin[h] if i != tau_[h]) == 1) #courier_arrives_destination_from_delivery_or_origin(08)

    for j in pickup_delivery:
        for h in H:
            solver.Add(solver.Sum(x[i, j, h] for i in V if i != j) - solver.Sum(x[j, i, h] for i in V if i != j) == 0) #courier_arrives_leaves_droppoff(09)
    
    for (i, j) in E:
        for h in H:
            solver.Add(S[i, h] + s[i] + t[i, j] <= S[j, h] + (1 - x[i, j, h]) * M) #courier_follows_matched_paths(10)/(34)

    for i in V:
        for h in H:
            solver.Add(S[i, h] >= a[i]) 
            solver.Add(S[i, h] <= b[i]) #pickup_deliver_timewindow(11)

    for i in A:
        for h in H:
            solver.Add(S[i, h] <= S[n + i, h]) #pickup_before_delivery(12)

    for (i, j) in E:
        for h in H:
            solver.Add(L[i, h] + l[j] <= L[j, h] + (1 - x[i, j, h]) * M) #capacity_for_next_loading(13)

    for i in V:
        for h in H:
            solver.Add(L[i, h] <= R[h]) #capacity_constraint(14)

    for h in H:
        solver.Add(L[origin[h], h] == 0) #origin_destination_empty_load(15)(1)
        solver.Add(L[destination[h], h] == 0) #origin_destination_empty_load(15)(2)

    # Linearization Constraints
    for (i, j) in E:
        for h in H:
            if i!=j:
                solver.Add(p[i, j, h] <= M * x[i, j, h]) #big M pricing (23)
    for (i, j) in E:
        for h in H:            
            solver.Add(c[i, j, h] <= M * x[i, j, h]) #big M compensation (24)
   
    for (i, j) in E:
        for h in H:
            if i != j:
                solver.Add(p[i, j, h] <= M * x[i, j, h])  # Falls x=0, dann ist p=0
                solver.Add(p[i, j, h] >= 0)  # p darf nicht negativ sein

    for (i, j) in E:
        for h in H:
            if i != j:
                solver.Add(c[i, j, h] <= M * x[i, j, h])  # Falls x=0, dann ist p=0
                solver.Add(c[i, j, h] >= 0)  # p darf nicht negativ sein
    
    # additional constraints
    for i in tau[1:]:
        solver.Add(solver.Sum(x[i, j, h] for h in H for j in V if i != j) == 1) #additional_constr (25)
    
    for h in H:
        solver.Add(solver.Sum(x[i, j, h] for i in tau_[1:] for h in H for j in V if i!= j) == 0) #additional_constr (26)

    # solve model
    status = solver.Solve()

    if status == pywraplp.Solver.OPTIMAL:
        optimal_x = {(i, j, h): x[i, j, h].solution_value() for (i, j) in E for h in H}
        optimal_S = {(i, h): S[i, h].solution_value() for i in V for h in H}
        optimal_L = {(i, h): L[i, h].solution_value() for i in V for h in H}
        optimal_z = {i: z[i].solution_value() for i in A}
        optimal_p = {(i, j, h): p[i, j, h].solution_value() for (i, j) in E for h in H}
        optimal_c = {(i, j, h): c[i, j, h].solution_value() for (i, j) in E for h in H}

        OF_p = solver.Objective().Value()
        Runtime1 = solver.WallTime()
        
        return OF_p, optimal_L, optimal_S, optimal_c, optimal_p, optimal_x, optimal_z, Runtime1
    else:
        return None




# data from instance
def read_data_create_sets(data1):
    delimiter=';'
    df=pd.read_csv(data1, delimiter=delimiter, header=0, parse_dates=['Time a', 'Time b'], date_format ="%H:%M") #, date_parser=pd.to_datetime
    dfdistricts=pd.read_csv(data2, delimiter=delimiter, header=0)
    m=int(df.at[0,"m"])
    n=int(df.at[0,"n"])
    if type(df.at[0,"WTP_Faktor"])==np.float64:
        WTP_Faktor=df.at[0,"WTP_Faktor"]
    if type(df.at[0,"WTP_Faktor"])== str:
        WTP_Faktor=df.loc[0:0,"WTP_Faktor"].str.replace(',','.').astype(float)[0]
    if type(df.at[0,"ETP_Faktor"])==np.float64:
        ETP_Faktor=df.at[0,"ETP_Faktor"]
    if type(df.at[0,"ETP_Faktor"])== str:
        ETP_Faktor=df.loc[0:0,"ETP_Faktor"].str.replace(',','.').astype(float)[0]
    Speed=int(df.at[0,"Speed"])
    start_rowpu=0                                                        
    start_rowdel= start_rowpu + n                                          
    start_rowdriver= start_rowdel + n
    A=df.loc[start_rowpu:start_rowdel-1,'i'].astype(int).tolist()
    A_=df.loc[start_rowdel:start_rowdriver-1,'i'].astype(int).tolist()
    AllX=df.loc[start_rowpu:start_rowdriver+2*m-1,'Long'].str.replace(',','.').astype(float).tolist()
    AllX.insert(0,0.0)
    AllY=df.loc[start_rowpu:start_rowdriver+2*m-1,'Lat'].str.replace(',','.').astype(float).tolist()
    AllY.insert(0,0.0)
    R2=df.loc[start_rowdriver:start_rowdriver+m-1,'R'].astype(int).tolist()
    R2.insert(0,0)
    district_names={idx: value for idx, value in enumerate (dfdistricts.loc[0:14,'Name'].astype(str), start=0)}
    district_coordx=dfdistricts.loc[start_rowpu:14,'Lat'].str.replace(',','.').astype(float).tolist()
    district_coordy=dfdistricts.loc[start_rowpu:14,'Long'].str.replace(',','.').astype(float).tolist()
    l={idx: int(value) for idx, value in enumerate (df.loc[start_rowpu:start_rowdriver+2*m-1,'l'].astype(int), start=1)}
    a={idx: int(value) for idx, value in enumerate (df.loc[start_rowpu:start_rowdriver+2*m-1,'a'].astype(int), start=1)}
    b={idx: int(value) for idx, value in enumerate (df.loc[start_rowpu:start_rowdriver+2*m-1,'b'].astype(int), start=1)}
    s={idx: int(value) for idx, value in enumerate (df.loc[start_rowpu:start_rowdriver+2*m-1,'s'].astype(int), start=1)}
    H = list(range(1, m + 1)) 
    TW_LB = {idx: pd.to_datetime(value, format="%H:%M:%S").strftime(time_format_output) 
         for idx, value in enumerate(df.loc[start_rowpu:start_rowdriver+2*m-1, 'Time a'], start=1)}

    TW_UB = {idx: pd.to_datetime(value, format="%H:%M:%S").strftime(time_format_output) 
         for idx, value in enumerate(df.loc[start_rowpu:start_rowdriver+2*m-1, 'Time b'], start=1)}
    R={h:R2[h] for h in H}
    tau=[2*n+h for h in H]                                                     # origin nodes of couriers
    tau.insert(0,0)
    tau_=[2*n + m +h for h in H]                                               # destination nodes of couriers
    tau_.insert(0,0)
    ZF=[b[i+m]-a[i] for i in tau[1:]]
    V= A + A_ +tau[1:] + tau_[1:]     
    pickup_delivery= A+A_                                                      # all pickup and delivery nodes
    destination={(h): tau_[h] for h in H}                                      # Set of destinations in dependence of driver h
    origin={(h): tau[h] for h in H}                                            # Set of origins in dependence of driver h
    pickup_destination= {h: A+[destination[h]] for h in H}                     # Unificated set of pickups and destinations in dependence of driver h    
    delivery_origin={h: [origin[h]]+A_ for h in H}                             # Unificated set of origins and drop-offs in dependence of driver h    
    delivery_destination={h: A_+[destination[h]] for h in H}
    Vh={(h): [origin[h]]+A+A_+[destination[h]] for h in H }                    # Set of all nodes in dependance of driver h
    origin_pickup_delivery={h: [origin[h]]+A+A_ for h in H}
    pickup_delivery_destination={h: pickup_delivery+[destination[h]] for h in H}
    origin_delivery_destination=A_+tau[1:]+tau_[1:] 
    E=[(i, j) for i in V for j in V if i!=j ]                                  # All edges except edges from one node to the same node back
    E1={h:[(i,j) for i in Vh[h] for j in Vh[h] if i!=j] for h in H}
    d={(i,j):haversine((AllX[i],AllY[i]),(AllX[j],AllY[j])) for i,j in E}      # travel distance for link i-->j
    d2={i:haversine((AllX[i],AllY[i]),(AllX[i+n],AllY[i+n])) for i in A}     
    d21={i:0 for i in origin_delivery_destination}
    d2.update(d21)
    t={(i,j):Speed*d[i,j] for i,j in E}
    
    
    return WTP_Faktor, ETP_Faktor, H, R, tau, tau_, V, pickup_delivery, ZF, TW_LB, TW_UB, destination, origin, pickup_destination, delivery_origin, delivery_destination, Vh, origin_pickup_delivery, pickup_delivery_destination, E, E1, d, d2, t, m, n, A, A_, a, b, l, s, R2, AllX, AllY, Speed, district_names, district_coordx, district_coordy 
all_data=read_data_create_sets(data1)
WTP_Faktor, ETP_Faktor, H, R, tau, tau_, V, pickup_delivery, ZF, TW_LB, TW_UB, destination, origin, pickup_destination, delivery_origin, delivery_destination, Vh, origin_pickup_delivery, pickup_delivery_destination, E, E1, d, d2, t, m, n, A, A_, a, b, l, s, R2, AllX, AllY, Speed, district_names, district_coordx, district_coordy  = all_data

# Machine learning for the WTP and ETP values
def create_wtp_etp():
    DP = 150                                                                   # Number of data points                          
    maxWTP=10                                                                  # Upper limit for the WTP corridor
    minWTP=5                                                                   # Lower limit for the WTP corridor
    wtp=dict()
    innerstädtisch=[]
    außerhalb=[]
    distance = {i:d2[i] for i in A}
    for i in A:                                                                # Differentiation between the inner and outer areas of Berlin's city center for the pick-up nodes
        if 13.32 <= AllX[i] <= 13.47 and 52.48 <= AllY[i] <= 52.525:
            base = int((maxWTP-minWTP)/2+minWTP)                               # Calculation of the base value in the inner area
            innerstädtisch.append(i)
        else:
            base = round((maxWTP-minWTP)/2+minWTP,0)                           # Calculation of the base value in the outer range
            außerhalb.append(i)
        for _ in range(DP):                                                    # Simulating the data points with the WTP variations
            wtp_value = base  + np.random.normal(0.2, 0.3)
            while wtp_value < minWTP or wtp_value > maxWTP:                    # Compliance with the WTP corridor
                wtp_value = base +  np.random.normal(0.1, 0.2)
            if i in wtp:
                wtp[i].append(wtp_value)
            else:
                wtp[i] = [wtp_value]
    distance_list = []
    wtp_list = []
    customer_list = []
    for customer_id, i in enumerate(A):                                        # Sorting of data points by customer number
        for dp in range(DP):
            distance_list.append(distance[i])
            wtp_list.append(wtp[i][dp])
            customer_list.append(i)
    # 'Historical data' created
   # Create the DataFrame
    dfML_WTP = pd.DataFrame({                                                  # Creation of the data frame with the customer numbers, the order distances and the simulated WTP values
        'customer_id': customer_list,
        'distance': distance_list,
        'WTP': wtp_list
        })
    # Split data into training data, test data and validation data
    train_data_WTP, test_data_WTP = train_test_split(dfML_WTP, test_size=0.2, stratify=dfML_WTP['customer_id'], random_state=42)
    train_data_WTP, val_data_WTP = train_test_split(train_data_WTP, test_size=0.2, stratify=train_data_WTP['customer_id'], random_state=42)
    test_data_WTP_avg = test_data_WTP.groupby('customer_id').mean().reset_index()
    # Conversion to Tensors
    train_tensors = {}
    unique_customers = train_data_WTP['customer_id'].unique()
    for customer_id in unique_customers:
        sub_data = train_data_WTP[train_data_WTP['customer_id'] == customer_id]
        train_distance = torch.tensor(sub_data['distance'].values, dtype=torch.float32)
        train_wtp = torch.tensor(sub_data['WTP'].values, dtype=torch.float32)
        train_tensors[customer_id] = {
            'distance': train_distance,
            'wtp': train_wtp
            }
    SteigungWTP=maxWTP/max(distance_list)                                      # Step length for the beta parameter 
    def Bayesian_model1(train_distance, train_wtp):                            # Creation of the Bayesian model
        # Prior distribution
        alpha = pyro.sample("alpha", dist.Normal(0.0, 0.01))
        beta_distance = pyro.sample("beta_distance", dist.Normal(SteigungWTP, 0.01))
        sigma = pyro.sample("sigma", dist.HalfNormal(0.01))
        # Likelihood
        mu = alpha + beta_distance * train_distance 
        with pyro.plate("data", len(train_wtp)):
            pyro.sample("obs", dist.Normal(mu, sigma), obs=train_wtp)
    posterior_samples_dict = {}
    predictions_dict = {}
    # Create Predictive-Object
    guide = AutoDiagonalNormal(Bayesian_model1)
     # Training the model and making predictions for each customer
    for customer_id, tensors in train_tensors.items():
        train_distance = tensors['distance']
        train_wtp = tensors['wtp']
        # Bayesian inference
        nuts_kernel = NUTS(Bayesian_model1)
        mcmc = MCMC(nuts_kernel, num_samples=150, warmup_steps=150)            # Number of samples and warm-up steps for the accuracy of the prediction of the A posteriori distribution
        mcmc.run(train_distance, train_wtp)                                    # Markov-Monte-Carlo method to draw samples for the A posteriori distribution
        # Posterior Samples
        posterior_samples = mcmc.get_samples()
        posterior_samples_dict[customer_id] = posterior_samples
        def predictive_model(train_distance, alpha=posterior_samples["alpha"], beta_distance=posterior_samples["beta_distance"], sigma=posterior_samples["sigma"]):
            mu = alpha + beta_distance * train_distance                        # A posteriori distribution is used for the prediction
            return pyro.sample("WTP", dist.Normal(mu, sigma))
        # Prediction
        predictive1 = Predictive(predictive_model, posterior_samples=posterior_samples, num_samples=150)
        new_distance1 = torch.tensor(test_data_WTP_avg[test_data_WTP_avg['customer_id'] == customer_id]['distance'].values, dtype=torch.float32)
        samples = predictive1(new_distance1)
        predictions_dict[customer_id] = samples
    # Preparation Output of WTP mean value in readable format for the pricing model
    wtp_mean_dict = {}
    wtp_std_dict = {}
    for customer_id, samples in predictions_dict.items():
        wtp_mean = samples["WTP"].mean(dim=0)
        wtp_std = samples["WTP"].std(dim=0)
        wtp_mean_dict[customer_id] = wtp_mean
        wtp_std_dict[customer_id] = wtp_std
    WTP={}
    for i, wtp_mean_values in wtp_mean_dict.items():
        WTP[i] = float(wtp_mean_values.mean().item())
                                        #ETP
    maxETP=1                                                                   # Upper limit for the ETP corridor
    minETP=0.5                                                                 # Lower limit for the ETP corridor
    etp={h:[] for h in H}
    innerstädtisch2=[]
    außerhalb2=[]
    for h in H:
        if 13.32 <= AllX[h + 2*n] <= 13.47 and 52.48 <= AllY[h + 2*n] <= 52.525:# Differentiation between the inner and outer areas of Berlin's city center for the pick-up nodes
            innerstädtisch2.append(h)
            base = int(((maxETP-minETP)/2+minETP)*10)/10                       # Calculation of the base value in the inner area
        else:
            außerhalb2.append(h)
            base = round((maxETP-minETP)/2+minETP,1)                           # Calculation of the base value in the outer area
        for _ in range(DP):                                                    # Simulating the data points with the ETP variations
            etp_value = base + np.random.normal(0.1, 0.2)
            while etp_value < minETP or etp_value > maxETP:                    #  Compliance with the ETP corridor
                etp_value = (base + np.random.normal(0.1, 0.2))
            if h in etp:
                etp[h].append(etp_value)
            else:
                etp[h] = [etp_value]
    # Datensatz distance3 von den Startknoten zu den Zielknoten erstellen                                
    distance_driver={h:d[origin[h],destination[h]]for h in H}
    distance_list3 = []
    etp_list = []
    driver_list = []
    for driver_id, h in enumerate(H):                                          # Sorting the data points by driver number 
        for dp in range(DP):
            distance_list3.append(distance_driver[h])
            etp_list.append(etp[h][dp])
            driver_list.append(h) 
    dfML_ETP = pd.DataFrame({                                                  # Creation of the data frame with the driver numbers, the route distances from start to finish nodes and the simulated ETP values
        'driver_id': driver_list,
        'distance3': distance_list3,
        'ETP': etp_list
        })
    # Split data into training data, test data and validation data
    train_data_ETP, test_data_ETP = train_test_split(dfML_ETP, test_size=0.2, stratify=dfML_ETP['driver_id'], random_state=42)
    train_data_ETP, val_data_ETP = train_test_split(train_data_ETP, test_size=0.2,stratify=train_data_ETP['driver_id'], random_state=42)
    test_data_ETP_avg = test_data_ETP.groupby('driver_id').mean().reset_index()
    # Conversion to tensors
    train_tensors2 = {}
    unique_drivers = train_data_ETP['driver_id'].unique()
    for driver_id in unique_drivers:
        sub_data2 = train_data_ETP[train_data_ETP['driver_id'] == driver_id]
        train_distance3 = torch.tensor(sub_data2['distance3'].values, dtype=torch.float32)
        train_etp = torch.tensor(sub_data2['ETP'].values, dtype=torch.float32)
        train_tensors2[driver_id] = {
            'distance3': train_distance3,
            'etp': train_etp
            }
    SteigungETP=float(-(maxETP-minETP)/(max(distance_list3)-min(distance_list3)))  # Step length for the beta parameter  
    def Bayesian_model2( train_distance3, train_etp):                          # Creation of the Bayesian model
        # Prior Distribution
        alpha2 = pyro.sample("alpha", dist.Normal(maxETP, 0.001))
        beta_distance3 = pyro.sample("beta_distance3", dist.Normal(SteigungETP, 0.001))
        sigma2 = pyro.sample("sigma", dist.HalfNormal(0.001))
        # Likelihood
        mu2 = alpha2 + beta_distance3 * train_distance3 
        with pyro.plate("data", len(train_etp)):
            pyro.sample("obs", dist.Normal(mu2, sigma2), obs=train_etp)
    posterior_samples_dict2 = {}
    predictions_dict2 = {}
    guide = AutoDiagonalNormal(Bayesian_model2)
    for driver_id, tensors2 in train_tensors2.items():
        train_distance3 = tensors2['distance3']
        train_etp = tensors2['etp']
        # Bayessche Inferez
        nuts_kernel = NUTS(Bayesian_model2)
        mcmc2 = MCMC(nuts_kernel, num_samples=150, warmup_steps=150)           # Number of samples and warm-up steps for the accuracy of the prediction of the A posteriori distribution
        mcmc2.run(train_distance3, train_etp)                                  # Markov-Monte-Carlo method to draw samples for the A posteriori distribution
        # Posterior Samples
        posterior_samples2 = mcmc2.get_samples()
        posterior_samples_dict2[driver_id] = posterior_samples2
        def predictive_model2(train_distance3, alpha2=posterior_samples2["alpha"], beta_distance3=posterior_samples2["beta_distance3"], sigma2=posterior_samples2["sigma"] ):
            mu2 = alpha2 + beta_distance3 * train_distance3                    # A posteriori distribution is used for the prediction
            return pyro.sample("ETP", dist.Normal(mu2, sigma2))
        # Prediction
        predictive2 = Predictive(predictive_model2, posterior_samples=posterior_samples2, num_samples=150)
        new_distance3 = torch.tensor(test_data_ETP_avg[test_data_ETP_avg['driver_id'] == driver_id]['distance3'].values, dtype=torch.float32)
        samples2 = predictive2(new_distance3)
        predictions_dict2[driver_id] = samples2
    # Preparation of the output of the ETP averages
    etp_mean_dict={}
    etp_std_dict={}
    for driver_id, samples2 in predictions_dict2.items():
        etp_mean = samples2["ETP"].mean(dim=0)
        etp_std = samples2["ETP"].std(dim=0)
        etp_mean_dict[driver_id] = etp_mean
        etp_std_dict[driver_id] = etp_std
    # Output of ETP mean values in readable format for the pricing model
    ETP={}
    for h, etp_mean_value in etp_mean_dict.items():
        ETP[h] = float(etp_mean_value.mean().item())
    return WTP, wtp_list, dfML_WTP, ETP,  etp_list,  dfML_ETP
ML=create_wtp_etp()
WTP, wtp_list, dfML_WTP, ETP, etp_list, dfML_ETP=ML

# Running Optimization
result = pricing_model(n, m, A, A_, H, WTP, ETP, R, d2, V, d, t, s, l, a, b, M)# running the function for optimization model
if result is not None:                                                         # print results when optimization successful  
    OF_p, optimal_L, optimal_S, optimal_c, optimal_p, optimal_x, optimal_z, Runtime1 = result      
else:
    print("Optimization failed.")

    
#extract variables
def results():
    active_arcs=[(i, j, h) for i, j in E for h in H if optimal_x[i, j, h]>0.99] 
    active_prices=[(i, j, h) for i, j in E for h in H if optimal_p[i, j, h]>0.99]
    active_costs=[(i, j, h) for i, j in E for h in H if optimal_c[i, j, h]>0.099]   
    active_load=[(i, h) for i in V for h in H  if optimal_L[i, h]>0.99] 
    active_z=[i for i in A if optimal_z[i]>0.99] 
    active_Start=[(i, h) for i in V for h in H if optimal_S[i, h]>0.99]
    td=sum(optimal_x[i, j, h]*d[i,j] for h in H for i in origin_pickup_delivery[h] for j in pickup_delivery  if i!=j)
    Fahrerstopps={h:sum(optimal_x[i,j,h] for i,j in E) for h in H}
    Multistop_touren={h:Fahrerstopps[h] for h in H}
    Touren={h:[(i,j) for i,j in E if optimal_x[i,j,h]>0.99 ] for h in H}
    for h in H:
        if Fahrerstopps[h]>=2:
            Multistop_touren[h]=(Fahrerstopps[h]-1)/2
        else:
            del Multistop_touren[h]
            del Touren[h]
    active_driver=[h for h in Touren if Touren[h]!=[] ]
    possible_matches= len(A)
    matches=len(A)-sum(optimal_z[i] for i in A)
    used_driver=sum(optimal_x[i,j,h] for i in A_ for j in tau_[1:] for h in H)
    active_arcs2D=list()
    for a in active_arcs:
        active_arcs2D.extend([a[:2]])
    active=list()
    for (i,j) in active_arcs2D:
        if i in tau[1:] and j in tau_[1:]:
            pass
        else:
            active.append((i,j))
    chosen_c={(i,j,h):optimal_c[i, j, h] for i,j in E for h in H if optimal_c[i, j, h]>0.099}
    chosen_cost={h:chosen_c[i,j,h] for i,j,h in chosen_c}
    chosen_p={(i):round(optimal_p[i, j, h],2) for i,j in E for h in H if optimal_p[i, j, h]>0.099}
    opt_request_p={i:round(chosen_p[i]*d2[i],2) for i in chosen_p}
    opt_driver_c={(h):round(chosen_c[i,j,h],2) for i,j,h in chosen_c}
    Einnahmen=round(sum(optimal_p[i,j,h]*d2[i] for i,j in E for h in H),2)
    Einnahmen_je_Tour={h:round(sum(optimal_p[i,j,h]*d2[i] for i,j in E ),2)for h in H}
    Ausgaben=round(sum(optimal_c[i,j,h]*d[i,j] for i,j in E for h in H),2)
    Ausgaben_je_Tour={h:round(sum(optimal_c[i,j,h]*d[i,j] for i,j in E ),2)for h in H}
    Marge_je_Tour={h:Einnahmen_je_Tour[h]-Ausgaben_je_Tour[h] for h in active_driver}
    aktive_Auftraege=[i for i in chosen_p]
    Einnahmen_je_Auftrag={i:d2[i]*chosen_p[i] for i in aktive_Auftraege}
    Fahrerdistanz_je_Auftrag={i:d[i,j]+d[a,i] for i in aktive_Auftraege for j in pickup_delivery for h in H for a in origin_pickup_delivery[h] if (i,j) in active and (a,i) in active }	
    Ausgaben_je_Auftrag={i:Fahrerdistanz_je_Auftrag[i]*chosen_c[i,j,h] for i,j,h in chosen_c if i in aktive_Auftraege}
    Marge_je_Auftrag={i:(Einnahmen_je_Auftrag[i]-Ausgaben_je_Auftrag[i]) for i in chosen_p} 
    Kantenkosten={(i,j):chosen_c[i,j,h] for h in active_driver for (i,j) in Touren[h] if j not in tau_[1:]}
    print("total distance from origin to dropoff=", td)
    print("active_arcs (x):", active_arcs)
    print("active_z (z):", active_z)
    print("Anzahl Aufträge je Fahrer:", Multistop_touren)
    print("Anzahl möglicher matches=",possible_matches)
    print("Anzahl erfolgreicher matches=", matches)
    print("Anzahl genutzter Fahrer:", used_driver)
    print("Folgende Fahrer werden eingesetzt:", active_driver)
    print("ZFW (Marge Platform Provider in €):", round(OF_p,2))
    print("Marge je Auftrag:", [Marge_je_Auftrag])
    for i in chosen_p:
        print("WTP bei Auftrag", [i], "=", WTP[i], "gewählter Preis für Auftrag", [i], "=", opt_request_p[i], "gewählter Preis je km =", round(chosen_p[i],2) )
    for h in active_driver:
        print("ETP for driver", [h], "=", ETP[h], "choosen cost per km =", opt_driver_c[h] )
    print("Einnahmen=", Einnahmen, "Ausgaben=", Ausgaben)
    return active_arcs, active_prices, active_costs, active_load, active_z, active_Start, Touren, active_driver, active_arcs2D, active, chosen_c, chosen_cost, chosen_p, opt_request_p, opt_driver_c, Einnahmen, Ausgaben, used_driver, possible_matches, matches, Kantenkosten, Marge_je_Auftrag, Marge_je_Tour, aktive_Auftraege
result2 = results()
active_arcs, active_prices, active_costs, active_load, active_z, active_Start, Touren, active_driver, active_arcs2D, active, chosen_c, chosen_cost, chosen_p, opt_request_p, opt_driver_c, Einnahmen, Ausgaben, used_driver, possible_matches, matches, Kantenkosten, Marge_je_Auftrag, Marge_je_Tour, aktive_Auftraege = result2

def variables_part2():
    
    active_nodes=[]
    for a in active:
        active_nodes.extend(a[:1])
        active_nodes.extend(a[1:])
    active_nodes=list(dict.fromkeys(active_nodes))
    Touren_neu={h:(tuple) for h in Touren}
    for h in Touren:
        edgars=[(i,j) for (i,j) in Touren[h]]
        GTouren = nx.DiGraph()
        GTouren.add_edges_from(edgars)
        longest_path = nx.dag_longest_path(GTouren)
        graph_after = list(zip(longest_path[:-1], longest_path[1:]))
        Touren_neu[h]=graph_after
    Touren_sorted={h:list() for h in Touren_neu}
    for h in Touren_neu:
        liste=[]
        for a in Touren_neu[h]:
            list(a)
            liste.append(a[:1])
        liste.append(a[1:])    
        liste=[i[0] for i in liste]
        Touren_sorted[h]=liste
    opt_S_driver={(i,h):optimal_S[i,h]  for h in Touren_sorted for i in  Touren_sorted[h]}
    Starting_times={i:int(optimal_S[i,h]) for h in Touren_sorted for i in Touren_sorted[h] }
    Starting_times_formatted={}
    for key, value in Starting_times.items():
        time_obj = timedelta(minutes=value)
        time_str = (datetime.min + time_obj).strftime('%H:%M')
        Starting_times_formatted[key] = time_str
    activitis=list()
    for h in H:
        if h not in active_driver:
            not_driving=h*-1
            activitis.append(not_driving)
        else:
            activitis.append(h)
    activitis.insert(0,0)
    used_origins={h:origin[h] for h in origin if h in active_driver}
    TW_LB_driver={h: str for h in used_origins}
    for h,i in used_origins.items():
        TW_LB_driver[h]=TW_LB[i]
    used_destinations={h:destination[h] for h in destination if h in active_driver}
    TW_UB_driver={h: str for h in used_destinations}
    for h,i in used_destinations.items():
        TW_UB_driver[h]=TW_UB[i]
    active_t={i:t[i,j] for i,j in active}
    print("Tour je Fahrer:", Touren_sorted)
    used_ETP={h:ETP[h] for h in active_driver}
    used_WTP={i:WTP[i] for i in active_nodes if i in A}
    Auftrag_Fahrer={node:driver for driver in Touren_sorted for node in Touren_sorted[driver]}
    Senders_surplus=round(sum(WTP[i] for i in aktive_Auftraege)-Einnahmen,2) 
    Couriers_surplus=round(Ausgaben-sum(ETP[h]*d[i,j]*optimal_x[i,j,h] for i,j in E for h in H if i in origin_pickup_delivery[h] and j in pickup_delivery),2)
    Umweg={(h): d[Touren_sorted[h][-2],destination[h]] for h in Touren_sorted }
    Umwege_summe=round(sum(Umweg[h]for h in Umweg),2)
    ges_km_je_Tour= {h: round(sum(d[i] for i in Touren_neu[h]),2) for h in Touren_neu}
    origin_dest_km={h: round(d[origin[h],destination[h]],2) for h in H}
    Umweg_ges_km=round(Umwege_summe/sum(ges_km_je_Tour[h] for h in Touren_sorted ),2)
    Umweg_Tour_Verhältnis={h:round(Umweg[h]/ges_km_je_Tour[h],2) for h in Touren_sorted}
    km_Auftrag={i:d[i,i+n] for i in A}
    Umweg_je_Tour={h: Umweg[h]/origin_dest_km[h] for h in Touren_sorted}
    ave_Umweg_je_Tour=np.mean(list(Umweg_je_Tour.values()))
    origin_dest_km2={h: round(d[origin[h],destination[h]],2) for h in H if h not in Touren_neu}  
    gefahrene_km=sum(ges_km_je_Tour[h] for h in ges_km_je_Tour)+sum(origin_dest_km2[h] for h in origin_dest_km2)
    Externalitaet=gefahrene_km/sum(origin_dest_km[h] for h in H)
    return Touren_neu, Touren_sorted,used_origins,TW_LB_driver,TW_UB_driver, used_destinations, activitis, opt_S_driver,  Touren, active_nodes, Starting_times, Starting_times_formatted, active_t, used_ETP, used_WTP, Auftrag_Fahrer, Senders_surplus, Couriers_surplus, Umwege_summe, Umweg, Umweg_ges_km, ges_km_je_Tour, origin_dest_km, Umweg_Tour_Verhältnis, km_Auftrag, Umweg_je_Tour, ave_Umweg_je_Tour, origin_dest_km2, gefahrene_km, Externalitaet
result3 = variables_part2()
Touren_neu, Touren_sorted,used_origins,TW_LB_driver,TW_UB_driver, used_destinations, activitis, opt_S_driver,  Touren, active_nodes, Starting_times, Starting_times_formatted, active_t, used_ETP, used_WTP, Auftrag_Fahrer, Senders_surplus, Couriers_surplus, Umwege_summe, Umweg, Umweg_ges_km, ges_km_je_Tour, origin_dest_km, Umweg_Tour_Verhältnis, km_Auftrag, Umweg_je_Tour, ave_Umweg_je_Tour, origin_dest_km2, gefahrene_km, Externalitaet = result3 


#draw graphs
def graph_map():
    file_path = os.path.join(os.getcwd(), "data_Berlin", "berlin_ortsteile.shp")
    districts = gpd.read_file(file_path)
    G=nx.DiGraph()
    G.add_nodes_from(V)
    G.add_edges_from(active)
    pos={i: (AllX[i],AllY[i])for i in V}
    fig1 = plt.figure(figsize=(16, 10))
    graph_ax = fig1.add_subplot(1, 1, 1,projection=ccrs.PlateCarree())
    graph_ax=plt.axes([0,0,1,1], projection=ccrs.PlateCarree())
    graph_ax.add_feature(cfeature.LAKES, edgecolor='black', alpha=0.5)
    graph_ax.add_feature(cfeature.RIVERS, edgecolor='black', alpha=0.9)
    for i in district_names:
        graph_ax.text(district_coordy[i], district_coordx[i], district_names[i], fontsize=8, color='grey', ha='center', va='top')
    districts.boundary.plot(ax=graph_ax, color='gray',  linewidth=1, alpha=0.5)
    graph_ax.add_feature(cfeature.NaturalEarthFeature(category='cultural', name='admin_0_boundary_lines_land', scale='10m', facecolor=None))
    graph_ax.set_extent([13.27, 13.51, 52.450, 52.56], crs=ccrs.PlateCarree())
    file_path2 = os.path.join(os.getcwd(), "data_Berlin", "Umweltzone_Berlin.shp")
    Umweltzone = gpd.read_file(file_path2)
    Umweltzone = Umweltzone.set_crs("EPSG:25833")
    target_crs = ccrs.PlateCarree()    
    Umweltzone = Umweltzone.to_crs(target_crs.proj4_init)
    for idx, polygon in Umweltzone.iterrows():
        graph_ax.add_geometries([polygon['geometry']], crs=ccrs.PlateCarree(), facecolor='green', edgecolor='black', alpha=0.1)
    options = {"edgecolors": "k", "node_size": 200, "alpha": 0.5}
    nx.draw_networkx_nodes(G, pos, nodelist=tau[1:], node_color="yellow", node_shape='H', **options)
    nx.draw_networkx_nodes(G, pos, nodelist=A, node_color="tab:orange", node_shape='o', **options)
    nx.draw_networkx_nodes(G, pos, nodelist=A_, node_color="tab:red", node_shape='o', **options)
    nx.draw_networkx_nodes(G, pos, nodelist=tau_[1:], node_color="tab:brown", node_shape='d', **options)
    nx.draw_networkx_labels(G, pos, labels={n:n for n in G}, font_size=8, font_color='k' )
    edges=list(G.edges())
    num_edges=len(edges)+1
    colormap= plt.colormaps['tab20']
    colores=[colormap(i/(num_edges)) for i in range(max(Touren)+1)]
    edgecolors={(a):colores[h] for h in Touren for (a) in Touren[h]}
    driver_color={(h):colores[h] for h in active_driver}
    edge_color_list = [edgecolors[a] for a in edges]
    nx.draw_networkx_edges(G, pos, edge_color=edge_color_list, alpha=1.0 ,width=1.5, arrows=True, arrowsize=12, ax=graph_ax, connectionstyle="arc3,rad=0.1")
    graph_ax.set_xticks([])
    graph_ax.set_yticks([])
    graph_ax.set_xlim(13.27, 13.51)
    graph_ax.set_ylim(52.450, 52.562)
    graph_ax.gridlines(draw_labels=True)
    graph_ax.add_feature(cfeature.LAND)
    plt.title("Tourenmap:IPIC_SP_p")
    plt.show()
    
    return edgecolors, colores, num_edges, driver_color, fig1
result4=graph_map()
edgecolors, colores, num_edges, driver_color, fig1 = result4


def graph_gantt():    
    Ending_times = {}
    Ending_times = {i: (Starting_times[i] + s[i] + active_t[i]) for i in Starting_times if i not in tau_[1:]}
    Ending_times2 = {i: Starting_times[i] for i in tau_[1:] if i in active_nodes}
    Ending_times.update(Ending_times2)
    Ending_times_formatted = {i: 0 for i in Starting_times}
    for key, value in Ending_times.items():
        time_obj = timedelta(minutes=value)
        time_str = (datetime.min + time_obj).strftime('%H:%M')
        Ending_times_formatted[key] = time_str
    start_3={i:Starting_times[i] + s[i]  for i in Starting_times if i not in tau_[1:]}
    start_32 = {i: Starting_times[i] for i in tau_[1:] if i in active_nodes}
    start_3.update(start_32)
    start3= {i: 0 for i in Starting_times}
    for key, value in start_3.items():
        time_obj = timedelta(minutes=value)
        time_str = (datetime.min + time_obj).strftime('%H:%M')
        start3[key] = time_str
    end_3={i:Starting_times[i] + s[i] + active_t[i]  for i in Starting_times if i not in tau_[1:]}
    end_32={i: Starting_times[i] for i in tau_[1:] if i in active_nodes}
    end_3.update(end_32)
    end3 = {i: 0 for i in Starting_times}
    for key, value in end_3.items():
        time_obj = timedelta(minutes=value)
        time_str = (datetime.min + time_obj).strftime('%H:%M')
        end3[key] = time_str
    df = []
    for h, i in Touren_sorted.items():
       for j in i:
            df.append({'task': j, 'Start': Starting_times_formatted[j], 'Finish': Ending_times_formatted[j], 'Resource': h, 'color': driver_color[h], 'start2':TW_LB_driver[h], 'end2':TW_UB_driver[h], 'start3':start3[j], 'end3':end3[j]})
    fig2, gantt_ax = plt.subplots(figsize=(16, 8))
    for item in df:
        start = datetime.strptime(item['Start'], '%H:%M')
        finish = datetime.strptime(item['Finish'], '%H:%M')
        duration = finish - start
        text_x2 = start+ duration/5
        text_y2 = item['Resource']
        gantt_ax.broken_barh([(start, duration)], (item['Resource'] - 0.25, 0.5), facecolors=item['color'], zorder=3, alpha=0.5)
        gantt_ax.annotate(item['task'], (text_x2, text_y2), xytext=(0, 0), textcoords='offset points',
                          ha='center', va='bottom', fontsize=9, color='black', zorder=5)
        start2 = datetime.strptime(item['start2'], '%H:%M')
        end2 = datetime.strptime(item['end2'], '%H:%M')
        start3= datetime.strptime(item['start3'], '%H:%M')
        end3 = datetime.strptime(item['end3'], '%H:%M')
        duration2 = end2 - start2
        duration3 = end3 - start3
        gantt_ax.broken_barh([(start2, duration2)], (item['Resource'] -0.25, 0.5), facecolors=item['color'], zorder=2, alpha=0.1)
        gantt_ax.broken_barh([(start3, duration3)], (item['Resource'] - 0.25, 0.5), facecolor='none', edgecolor='lightgrey', hatch='//', zorder=4)
    Tourdauer = mpatches.Patch(color='tab:red', label='Servicezeit am Knoten')
    Fahrer_ZF = mpatches.Patch(color='pink', label='Zeitfenster der Fahrer in hell')
    Fahrtzeit = mpatches.Patch(facecolor='none', hatch='//', edgecolor='grey', label='Fahrtzeit')
    Knoten_nummer = mpatches.Patch(color='white', label='1,2,.: Nr besuchter Knoten ')
    gantt_ax.legend(handles=[Tourdauer, Fahrer_ZF, Fahrtzeit, Knoten_nummer], loc='lower right')    
    plt.xlabel("Zeit")
    plt.ylabel("Fahrer")
    plt.title("Gantt:IPIC_SP_p")
    y_ticks = list(Touren_sorted.keys())
    y_labels = list(Touren_sorted.keys())  
    y_pos=list(Touren_sorted.keys())
    gantt_ax.set_yticks(y_pos, y_ticks)
    gantt_ax.set_yticklabels(y_labels) 
    x_ticks = pd.date_range(start=datetime(1900, 1, 1, 10, 0), end=datetime(1900, 1, 1, 20, 00), freq='h')
    x_labels = [time.strftime('%H:%M') for time in x_ticks]
    gantt_ax.set_xticks(x_ticks)
    gantt_ax.set_xticklabels(x_labels, rotation=45, ha='right')
    gantt_ax.grid(zorder=0)
    plt.show()
    return Ending_times, Ending_times_formatted, df, start3, fig2

result5 = graph_gantt()
Ending_times, Ending_times_formatted, df, start3, fig2 =result5
